In [1]:
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(device)

!nvidia-smi

cuda
Sat Dec  5 02:24:56 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [2]:
!pip install git+https://github.com/kernelmachine/allennlp.git@4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
!pip install transformers==2.4.1
!pip install pytorch-transformers==1.2.0
!git clone https://github.com/allenai/dont-stop-pretraining

!pip install adapter-transformers

  Cloning https://github.com/kernelmachine/allennlp.git (to revision 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7) to /tmp/pip-req-build-f85acw74
  Running command git clone -q https://github.com/kernelmachine/allennlp.git /tmp/pip-req-build-f85acw74
  Running command git checkout -q 4ae123d2c3bfb1ea3ce7362cb6c5bca3d094ffa7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 71kB 7.9MB/s 
     |████████████████████████████████| 133kB 17.2MB/s 
     |████████████████████████████████| 133kB 20.0MB/s 
     |████████████████████████████████| 51kB 7.1MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 266kB 20.6MB/s 
     |████████████████████████████████| 317kB 24.2MB/s 
     |████████████████████████████████| 245kB 32.9MB/s 
     |████████████████████████████████| 5.3MB 24.8MB/s 
     |████████████████████████████████| 30.9MB 180kB/s

In [3]:
!ls temp

ls: cannot access 'temp': No such file or directory


In [4]:
!git clone https://github.com/nsusanj/temp.git

Cloning into 'temp'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!ls temp/adapticons

modeling  new_train_requirements.txt  README.md


In [7]:
%cd temp/adapticons
%mkdir datasets
%cd datasets/
%mkdir citation
%cd citation

/content/temp/adapticons
/content/temp/adapticons/datasets
/content/temp/adapticons/datasets/citation


In [8]:
!curl -Lo train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl -Lo dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl -Lo test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   290k      0  0:00:01  0:00:01 --:--:--  290k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  35125      0 --:--:-- --:--:-- --:--:-- 35084
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0  44979      0 --:--:-- --:--:-- --:--:-- 44926


In [9]:
%cd ../../modeling

/content/temp/adapticons/modeling


In [10]:
!ls ../datasets/citation

dev.jsonl  test.jsonl  train.jsonl


In [11]:
!ls ../../../gdrive/MyDrive/downsample_sciee/_75_pct

checkpoint-1000  merges.txt		  tokenizer_config.json
checkpoint-500	 pytorch_model.bin	  training_args.bin
config.json	 special_tokens_map.json  vocab.json


In [13]:
!cp -r ../../../gdrive/MyDrive/downsample_sciee/_75_pct .

In [14]:
!ls

_75_pct		      new_train.py
download_glue.py      run_language_modeling_with_adapter_fusion.py
glue_training.py      run_language_modeling_with_adapters.py
jsonl_to_txt.py       runs
load_fusion_model.py  sample_pct_txt_file.py


In [16]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_75_tapt \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_75_pct \
  --metric macro

2020-12-04 02:50:41.073657: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/04/2020 02:50:42 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/04/2020 02:50:42 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_75_tapt', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_s

In [12]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_50_tapt \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_50_pct \
  --metric macro

2020-12-05 02:29:21.236760: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 02:29:23 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 02:29:23 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_50_tapt', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_s

In [13]:
!python new_train.py \
  --do_train \
  --do_eval \
  --data_dir ../datasets/citation/ \
  --max_seq_length 512 \
  --per_device_train_batch_size 16 \
  --gradient_accumulation_steps 1 \
  --learning_rate 2e-5 \
  --num_train_epochs 10 \
  --output_dir ../../../gdrive/MyDrive/downsample_newtrain/_25_tapt \
  --task_name citation \
  --do_predict \
  --load_best_model_at_end \
  --model_name_or_path ../../../gdrive/MyDrive/downsample_sciee/_25_pct \
  --metric macro

2020-12-05 02:41:38.851898: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
PyTorch: setting up devices
12/05/2020 02:41:40 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
12/05/2020 02:41:40 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir='../../../gdrive/MyDrive/downsample_newtrain/_25_tapt', overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=True, evaluate_during_training=False, evaluation_strategy=<EvaluationStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_s